<a href="https://colab.research.google.com/github/souravs17031999/private-ai/blob/master/encrypted_deep_learning_in_pysyft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project - XV
## Objective : Part - I :To implement encrypted computations using pysyft  functions/methods.
## Objective : Part - I :Create a encrypted database.

### Part - I

In [0]:
pip install syft

In [2]:
import torch as th
import syft as sy

hook = sy.TorchHook(th)
from torch import nn, optim

W0718 12:51:58.535861 139701015627648 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0718 12:51:58.559371 139701015627648 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



We have created three virtual workers here , bob , alice and secure_worker to simulate our process.

In [0]:
bob = sy.VirtualWorker(hook, id = "bob").add_worker(sy.local_worker)
alice = sy.VirtualWorker(hook, id = "alice").add_worker(sy.local_worker)
secure_worker = sy.VirtualWorker(hook, id = "secure_worker").add_worker(sy.local_worker)

Let's create our data.

In [0]:
x = th.tensor([1,2,3,4])
y = th.tensor([2,-1,1,0])

Let's say we firstly wanted to implement the secret sharing process so that there are randomly generated numbers , and since crypto_provider here is used only for system efficiency for deep learning systems when generating randomly large numbers.

In [0]:
x = x.share(bob, alice, crypto_provider = secure_worker)
y = y.share(bob, alice, crypto_provider = secure_worker)

We can see that , there are pointer tensors created for x and y for bob and alice.

In [7]:
x

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:7642784648 -> bob:46287031019]
	-> (Wrapper)>[PointerTensor | me:58627500828 -> alice:49566915719]
	*crypto provider: secure_worker*

In [8]:
y

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:41823787541 -> bob:69190081281]
	-> (Wrapper)>[PointerTensor | me:91802539315 -> alice:22111021685]
	*crypto provider: secure_worker*

Here , we can see randomly generated numbers for both tensors x, y

In [13]:
bob._objects 

{46287031019: tensor([1810776115422868363, 2592432799680637684, 3428359298948302875,
          830746408484362442]),
 69190081281: tensor([2433232717817796119,  386352628506133619, 2673491792552612824,
          878809810787705438])}

In [14]:
alice._objects


{22111021685: tensor([-2433232717817796117,  -386352628506133620, -2673491792552612823,
          -878809810787705438]),
 49566915719: tensor([-1810776115422868362, -2592432799680637682, -3428359298948302872,
          -830746408484362438])}

In [15]:
secure_worker._objects

{}

We will now start implementing operations using pysyft.

In [18]:
z = x + y
z
z.get()

tensor([3, 1, 4, 4])

In [19]:
z = x - y
z.get()

tensor([-1,  3,  2,  4])

In [20]:
z = x > y
z.get()

tensor([0, 1, 1, 1])

In [0]:
z = x < y

In [23]:
z = x == y
z.get()

tensor([0, 0, 0, 0])

In [24]:
z = x * y
z.get()

tensor([ 2, -2,  3,  0])

We can't just only this for integers , but also for floating numbers.

In [0]:
x = th.tensor([1,2,3,4])
y = th.tensor([2,-1,1,0])

x = x.fix_precision().share(bob, alice, crypto_provider=secure_worker)
y = y.fix_precision().share(bob, alice, crypto_provider=secure_worker)

In [27]:
x

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:23666081541 -> bob:35316031876]
	-> (Wrapper)>[PointerTensor | me:83754517187 -> alice:42948230833]
	*crypto provider: secure_worker*

In [28]:
z = x + y
z.get().float_precision()

tensor([3., 1., 4., 4.])

In [29]:
z = x - y
z.get().float_precision()

tensor([-1.,  3.,  2.,  4.])

In [30]:
z = x * y
z.get().float_precision()

tensor([ 2., -2.,  3.,  0.])

In [31]:
z = x > y
z.get().float_precision()

tensor([0., 1., 1., 1.])

In [32]:
z = x < y
z.get().float_precision()

tensor([1., 0., 0., 0.])

In [33]:
z = x == y
z.get().float_precision()

tensor([0., 0., 0., 0.])

### Part -2 